In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('Terry_Stops.csv')

/var/folders/k7/9snc3m492cjbwzx2tb46jfcm0000gn/T/ipykernel_59391/710233119.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Terry_Stops.csv')


In [8]:
df.info

<bound method DataFrame.info of       Subject Age Group   Subject ID     GO / SC Num  Terry Stop ID  \
0                1 - 17           -1  20170000317010         301638   
1               36 - 45   7727706299  20250000063255    63109655222   
2          56 and Above           -1  20190000057245         532414   
3               46 - 55   7729078044  20230000235837    51923259552   
4               46 - 55   7746702884  20210000151070    25601408632   
...                 ...          ...             ...            ...   
66431           18 - 25  12853348161  20200000123001    12853342426   
66432           36 - 45           -1  20180000290417         452430   
66433           36 - 45           -1  20180000377580         490218   
66434           18 - 25           -1  20160000094749         137815   
66435           36 - 45  11229884412  20250000287386    66338374093   

      Stop Resolution Weapon Type Officer ID  Officer YOB Officer Gender  \
0              Arrest         NaN      